In [ ]:
#Requirements


In [ ]:
#!pip freeze > requirements.txt

In [ ]:
#supporting import
import pandas as pd
import numpy as np
import os
import sys
from datasets import load_dataset
from pprint import pprint
import re
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk
from sklearn.utils import shuffle
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import tokenize
from operator import itemgetter
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
nlp = spacy.load("en_core_web_sm")
from transformers import T5ForConditionalGeneration,T5Tokenizer
pd.set_option("display.max_colwidth", -1)

In [ ]:
#extract sentences

def extract_sentences(file):
    content = nlp(str(file))
    return [x.text for x in content.sents]

outText = []
text_path = "/inputText"
files = (file for file in os.listdir(text_path)
         if os.path.isfile(os.path.join(text_path, file)))
for file in files:
  key_name = os.path.splitext(file)[0]
  mypdf = open(os.path.join(text_path, file), "r")
  T = mypdf.readlines()
  final = [i.replace('\r\n',' ').replace('b\r\n', '').replace('\n', '')  for i in T]
  outText.append(T)
  # newtext.append(final)

In [ ]:
sentText= extract_sentences(outText)
final = [i.replace("\n'"," ") for i in sentText]
#result = re.sub(r'http\S+', '', str(final))
newFinal =[re.sub("[\(%\[].*?[\)\n]]", " ", i) for i in final]
newFinal =[re.sub(r'http\S+', '', i) for i in newFinal]
df2 =pd.DataFrame(final, columns= ['sentences'])
df3 =pd.DataFrame(newFinal, columns= ['sentences'])

for i in range(0, len(df2)):
    df2.at[i, 'sentences_new'] = re.sub(r'\b(\d+\w{1,2})\b'," ",str(df2.loc[i,'sentences']))
    df2.at[i, 'sentences_new'] = re.sub(r'[\d+]'," ",str(df2.loc[i,'sentences_new']))
    df2.at[i, 'sentences_new'] = re.sub(r'[^\w\s]'," ",str(df2.loc[i,'sentences_new']))
    #df2.at[i, 'sentences_new'] = re.sub(r'\b\w{1}\b'," ",str(df2.loc[i,'sentences_new']))
df2['text'] = df2['sentences_new'].str.replace(r"\s*https?:'//\S+(\s+|$)", " ").str.strip()
df2['text2'] = df2['text'].str.replace(r"ufeff", " ").str.strip()


In [ ]:
#load default dataset and splits for pretraining
train_data = load_dataset('squad', split='train')
validate_data = load_dataset('squad', split='validation')
sample_validation_dataset = next(iter(validate_data))
df_train = pd.DataFrame(columns = ['context','answers', 'question'])
df_validation = pd.DataFrame(columns = ['context', 'answers', 'question'])
counter_train= 0
for index,val in enumerate(tqdm(train_data)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words > 5:
        df_train.loc[counter_train]= [passage] + [answer] + [question]
        counter_train = counter_train + 1

counter_val = 0
df_train = pd.DataFrame(columns=['passage','answer','question'])
for index,val in enumerate(tqdm(validate_data)):
    passage = val['context']
    question = val['question']
    answer = val['answers']['text'][0]
    no_of_words = len(answer.split())
    if no_of_words > 5:
        df_validation.loc[counter_val]= [passage] + [answer] + [question]
        counter_val = counter_val + 1



In [ ]:
train_save_path = 'model/t5/dataset/squad_t5_train.csv'
validation_save_path = 'model/t5/dataset/squad_t5_train.csv'
df_train.to_csv(train_save_path, index = False)
df_validation.to_csv(validation_save_path, index = False)

In [ ]:
import argparse
from itertools import chain
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
# from termcolor import colored
import textwrap
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
pl.seed_everything(42)

In [ ]:
#Load base model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [ ]:
#question generation class
import copy
class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, filepath, max_len_inp=512,max_len_out=96):
        self.path = filepath

        self.passage_column = "context"
        #self.answer = "answer"
        self.question = "question"

        # self.data = pd.read_csv(self.path)
        self.data = pd.read_csv(self.path,nrows=1000)

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.skippedcount =0
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for idx in tqdm(range(len(self.data))):
            passage,target = self.data.loc[idx, self.passage_column], self.data.loc[idx, self.question]

            input_ = "context: %s </s>" % (passage)
            target = "question: %s </s>" % (str(target))

            # get encoding length of input. If it is greater than self.max_len skip it
            test_input_encoding = self.tokenizer.encode_plus(input_,
                                        truncation=False,
                                        return_tensors="pt")

            length_of_input_encoding = len(test_input_encoding['input_ids'][0])


            if length_of_input_encoding > self.max_len_input:
              self.skippedcount = self.skippedcount + 1
              continue

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len_input, pad_to_max_length=True, return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len_output, pad_to_max_length=True,return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)


In [ ]:
train_dataset = QuestionGenerationDataset(t5_tokenizer,train_save_path)
validation_dataset = QuestionGenerationDataset(t5_tokenizer,validation_save_path)
train_sample = train_dataset[51]
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self,hparams, t5model, t5tokenizer):
        super(T5FineTuner, self).__init__()
        self.hparams = hparams
        self.model = t5model
        self.tokenizer = t5tokenizer


    def forward( self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )

         return outputs


    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_input_ids = batch["target_ids"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.hparams.batch_size,num_workers=4)

    def val_dataloader(self):
        return DataLoader(validation_dataset, batch_size=self.hparams.batch_size,num_workers=4)



    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer


In [ ]:
args_dict = dict(
    batch_size=4,
)

args = argparse.Namespace(**args_dict)

model = T5FineTuner(args,t5_model,t5_tokenizer)

trainer = pl.Trainer(max_epochs = 1, gpus=1,progress_bar_refresh_rate=30)

trainer.fit(model)

print ("Saving model")
save_path_model = 'model/t5/model/'
save_path_tokenizer = 'model/t5/tokenizer/'
model.model.save_pretrained(save_path_model)
t5_tokenizer.save_pretrained(save_path_tokenizer)

In [ ]:
from transformers import T5ForConditionalGeneration,T5Tokenizer
trained_model_path = '/model/t5/model/'
trained_tokenizer = '/model/t5/tokenizer/'
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)
#check device cude availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)


In [ ]:
# Generate questions with our data using trained model
context = df2['sentences_new']
text = "context: "+context
input_ids =[]
attention_mask = []
for x in (text):
  encoding = tokenizer.encode_plus(x,max_length =512, padding=True, return_tensors="pt")
  print (encoding.keys())
  input_ids.append(encoding["input_ids"].to(device))
  attention_mask.append(encoding["attention_mask"].to(device))
  input_ids, attention_mask = encoding["input_ids"].to(device),encoding["attention_mask"].to(device)
data = dict(zip(input_ids, attention_mask))

model.eval()
cqs = []

for x in (text):
  encoding = tokenizer.encode_plus(x,max_length =512, padding=True, return_tensors="pt")
  print (encoding.keys())
  input_ids, attention_mask = encoding["input_ids"].to(device),encoding["attention_mask"].to(device)
  beam_outputs = model.generate(
      input_ids=input_ids,attention_mask=attention_mask,
      max_length=72,
      early_stopping=True,
      num_beams=5,
      num_return_sequences=3
  )

  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      cqs.append(sent)


In [ ]:
# Post processing of generated questions
questions =[]
with open("CQs.txt", "w") as file:
    file.write(str(cqs))
with open("./CQs.txt","r",encoding="utf-8") as f:
  for line in f.readlines():
    questions.append(line)
new_questions = []
for i in (questions):
  x = str(i).split(":")
  x = str(i).replace("question", '')
  new_questions.append(x)
new_questions2 = str(new_questions).split(",")
dcqs =pd.DataFrame(new_questions2)
dcqs.columns = ['CQs']
dcqs['CQs'] = dcqs['CQs'].map(lambda x: x.replace(":", '').replace("'", '').replace('[', '').replace(']', '').replace('"', ''))
# dcqs.head()
new_questions = []
for i in (questions):
  x = str(i).split(":")
  x = str(i).replace("question", '')
  new_questions.append(x)
new_questions2 = str(new_questions).split(",")
dcqs =pd.DataFrame(new_questions2)
dcqs.columns = ['CQs']
dcqs['CQs'] = dcqs['CQs'].map(lambda x: x.replace(":", '').replace("'", '').replace('[', '').replace(']', '').replace('"', ''))
dcqs.head()
#new_questions

for i in range(0, len(dcqs)):
    dcqs.at[i, 'newCQs'] =re.sub(r"\s\([A-Z][a-z]+,\s[A-Z][a-z]?\.[^\)]*,\s\d{4}\)","",str(dcqs.loc[i,'CQs']))
with open("output.txt", "w") as f:
    f.write("\n".join(dcqs["newCQs"]))

In [ ]:
#Abstract pattern extraction
def mark_chunk(cq, spans, chunktype, offset, counter):
    for (start, end) in spans:  # for each span of EC/PC candidate
        cq = cq[:start - offset] + chunktype + str(counter) +\
            cq[end - offset:]  # substitute that candidate with EC/PC marker
        offset += (end - start) - len(chunktype) - 1  # check by how much the total length of CQ changed
    return cq, offset


def extract_EC_chunks(cq):
    """
        Find EC chunks and replace their occurences with EC tags
    """

    def _get_EC_span_reject_wh_starters(chunk):
        """
            By default, SpaCy treats question words (wh- pronouns starting questions: where, what,...)
            as nouns, so often if questions starts with wh- pronoun + noun (like: What software is the best?)
            the whole "what software" is interpreted as EC chunk - this function tries to fix that issue by
            omitting wh- word if EC candidate consists of multiple tokens, and first token in question is wh- word.
            The same issue occurs for "How" starter.
            Moreover, chunks extracted with SpaCy enclose words like 'any', 'some' which are important for us, so
            they shouldn't be substituted with 'EC' marker. Thus we remove such words if they prepend the EC.
            The result is returned as the span - the position of beginning of the fixed EC and position of end.
        """
        if (len(chunk) > 1 and
                (chunk[0].text.lower().startswith("wh")) or
                (chunk[0].text.lower() == 'how')):
            chunk = chunk[1:]

        if chunk[0].text.lower() in ['any', 'some', 'many', 'well', 'its']:
            chunk = chunk[1:]

        return (chunk.start_char, chunk.end_char)

    doc = nlp(cq)

    #  thins classified as ECs which shouldn't be interpreted that way
    rejecting_ec = ["what", "which", "when", "where", "who", "type", "types",
                    "kinds", "kind", "category", "categories", "difference",
                    "differences", "extent", "i", "we", "respect", "there",
                    "not", "the main types", "the possible types", "the types",
                    "the difference", "the differences", "the main categories"]

    counter = 1  # counter indicating current chunk id (EC1, EC2, ...)
    offset = 0   # if we replace for example "Weka" with 'EC1' then the new CQ will be shorter by one char the offset remembers by how much we have shortened the current CQ with EC markers, so the new substitutions can be applied in correct places.


    # we decided to treat qualities defined as adjectives in: How + Quality(adjective) + Verb as EC
    if (doc[0].text.lower() == 'how' and
            doc[1].pos_ == 'ADJ' and
            doc[2].pos_ == 'VERB'):

        start = doc[1].idx  # mark where quality starts
        end = start + len(doc[1])  # mark where quality ends

        cq, offset = mark_chunk(cq, [(start, end)], "EC", offset, counter)  # substitute quality with EC identifier
        counter += 1  # the next EC chunk should have a new, bigger identifier, for example EC2

    for chunk in doc.noun_chunks:  # for each EC chunk candidate detected
        (start, end) = _get_EC_span_reject_wh_starters(chunk)  # check where chunk begins and ends

        ec = cq[start - offset:end - offset]  # extract text of potential EC, apply offsets

        if ec.lower() in rejecting_ec:  # if it should be rejected - do nothing
            continue

        if "the thing" in ec and end - start > len("the thing"):
            cq = cq[:start - offset] + "EC" + str(counter) +\
                " EC" + str(counter + 1) + cq[end - offset:]
            counter += 2
            offset += (end - start) - 7
        else:
            cq, offset = mark_chunk(cq, [(start, end)], "EC", offset, counter)
            counter += 1

    if (doc[-2].pos_ == 'VERB' and doc[-3].text in ['are', 'is', 'were', 'was'] and doc[-1].text == '?') or (doc[-2].pos_ in ['ADJ', 'ADV'] and doc[-1].text == '?'):
        # if CQ ends with are/is/were/was + VERB + ? or the last token is ADJective or ADVerb, treat the
        # verb / adverb / adjective as EC
        # Which animals are endangered -> endangered is EC
        # Which animals are quick -> quick is EC
        if doc[-2].text.lower() not in rejecting_ec:
            start = doc[-2].idx
            end = start + len(doc[-2])

            cq, offset = mark_chunk(
                cq, [(start, end)], "EC", offset, counter)
            counter += 1

    return cq


def get_PCs_as_spans(cq):
    def _is_auxilary(token, chunk_token_ids):
        """
            Check if given token is an auxilary verb of detected PC.
            The auxilary verb can be in a different place than the main part
            of the PC, so pos-tag-sequence based rules don't work here.
            For example in "What system does Weka require?" - the main part
            of PC is the word 'required'. The auxilary verb 'does' is separeted
            from the main part by 'Weka' noun. Thus dependency tree is used
            to identify auxilaries.
        """
        if (token.head.i in chunk_token_ids and  # if dep-tree current token's parent (head) is somewhere inside the main part of PC represented as chunk_token_ids (sequence of numeric token identifiers)
                token.dep_ == 'aux' and  # if the dep-tree label on the edge between some word from main part of PC and current token is AUX (auxilary)
                token.i not in chunk_token_ids):  # if token is outside of detected main part of PC
            return True  # yep, it's auxilary
        else:
            return False

    def _get_span(group, doc):
        id_tags = group.split(",")
        ids = [int(id_tag.split("::")[0]) for id_tag in id_tags]
        aux = None
        for token in doc:
            if _is_auxilary(token, ids):
                aux = token

        return (doc[ids[0]].idx, doc[ids[-1]].idx + len(doc[ids[-1]]),
                aux)

    def _reject_subspans(spans):
        """
            Given list of (chunk begin index, chunk end index) spans,
            return only those spans that aren't subspans of any other span.
            For instance form list [(1,10), (2,5)], the second span
            will be rejected because it is a subspan of the first one.
        """
        filtered = []
        for i, span in enumerate(spans):
            subspan = False
            for j, other in enumerate(spans):
                if i == j:
                    continue

                if span[0] >= other[0] and span[1] <= other[1]:
                    subspan = True
                    break
            if subspan is False:
                filtered.append(span)
        return filtered

    doc = nlp(cq)

    """
        Transform CQ into a form of POS-tags with token sequence identifier.
        Each token is described with "{ID}::{POS_TAG}".
        Tokens are separated with ","
        Having that form, we can extract longest sequences of expected pos-tags
        using regexes. The extracted parts can be explored to collect identifiers
        of tokens, so we know where they are located in text.
        Ex: "Kate owns a cat" should be translated into: "1::NOUN,2::VERB,3::DET,4::NOUN"
    """
    pos_text = ",".join(
        ["{id}::{pos}".format(id=id, pos=t.pos_) for id, t in enumerate(doc)])

    regexes = [   # rules describing PCs
        r"([0-9]+::(PART|VERB),?)*([0-9]+::VERB)",
        r"([0-9]+::(PART|VERB),?)+([0-9]+::AD(J|V),)+([0-9]+::ADP)",
        r"([0-9]+::(PART|VERB),?)+([0-9]+::ADP)",
    ]

    spans = []  # list of beginnings and endings of each chunk
    for regex in regexes:  # try to extract chunks with regexes
        for m in re.finditer(regex, pos_text):
            spans.append(_get_span(m.group(), doc))  # get chunk begin and end if matched
    spans = _reject_subspans(spans)  # reject subspans
    return spans


def extract_PC_chunks(cq):
    rejecting_pc = ['is', '痴', 'are', 'was', 'do', 'does', 'did', 'were',
                    'have', 'had', 'can', 'could', 'categorise', 'regarding',
                    'is of', 'are of', 'are in', 'given', 'is there']

    offset = 0
    counter = 1

    for begin, end, aux in get_PCs_as_spans(cq):
        if cq[begin - offset:end - offset].lower() in rejecting_pc:
            continue

        spans = [(begin, end)]

        if aux:
            spans.insert(0, (aux.idx, aux.idx + len(aux)))

        cq, offset = mark_chunk(cq, spans, "PC", offset, counter)
        counter += 1

    return cq



In [ ]:
# post abstarction processing
genQuestion=[] 
splitQuestions =[]
patterns = []
pattern_candidates = {}
T5Cqs_patterns = []
path_to_output = "./output.txt"
with open(path_to_output, 'r') as f:
  for line in f.readlines():
    line= line.replace('?', '? ')
    line2 = line.split("?")
    line3= [p.join(' ?') for p in line2]
    line3 = [p.lstrip() for p in line3]
    genQuestion.append(line3)
genQuestionDF= pd.DataFrame(genQuestion).T
genQuestionDF2 = genQuestionDF.drop_duplicates(subset='questions', keep="first")
genQuestionDF.columns=['questions']

allList=[]
pat_candidates={}
for sublist in  genQuestion: 
  final = []
  for i in sublist:
    if(i==''):
      continue
    for item in i.split(','):
      pat = extract_EC_chunks(item)
      pat = extract_PC_chunks(pat)
      final.append(pat)
      pat_candidates[item]=(pat)
      
  allList.append(final)

In [ ]:
absWithCQs= pd.DataFrame(pat_candidates.items(), columns=['questions', 'patterns'])
absWithCQs = absWithCQs.iloc[:-1]


In [ ]:
# save to file for human review where necessary. you import back into the process afterwars here
absWithCQs.to_csv("/CQs/geneCQsNov.csv", sep='|')
#import CLaRO templates
templates= pd.read_csv("/CLaROv2.csv", sep = ";")
templates.columns = 'ID', 'patterns'
#check if present in CLaRO

def compare_patterns(absWithCQs, templates):
    check = pd.merge(absWithCQs, templates, on=['patterns'], how='left', indicator='Exist')
    check_in_both = check[check['Exist'] == 'both']
    check_in_cqPatterns_only = check[check['Exist'] == 'left_only']
    check_in__templates_only = check[check['Exist'] == 'right_only']
    #extract distict ptterns found in templates 
    distinct_patterns = check_in_both.drop_duplicates(subset='patterns', keep="first")
    return distinct_patterns, check_in__templates_only, check_in_cqPatterns_only, check_in_both

In [ ]:
#semantic similarity to check CQs that are the same and should be dropped and also find questions that not similar
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
CovidCorpus = list(set(list(genQuestionDF2['questions'])))
query_Corpus =list(set(list(genQuestionDF2['questions'])))
#Compute embedding for both lists
corpus_embed = model.encode(query_Corpus, convert_to_tensor=True)
covidSemanticCQs= {}
top_k = min(5, len(CovidCorpus))
for qry in query_Corpus:
    query_embed = model.encode(qry, convert_to_tensor=True, device=device)
    cos_scores = util.cos_sim(query_embed, corpus_embed)[0]
    top_results = torch.topk(cos_scores, k=top_k)
for score, idx in zip(top_results[0], top_results[1]):
    new_score= "{:.4f}".format(score)
    new_score= float(new_score)
    if  new_score <= 0.7500:
        covidSemanticCQs[qry]=[]
        covidSemanticCQs[qry].append({query_Corpus[idx]: (new_score)})
    QuesSimilar= pd.json_normalize([ { "firstCQs": key_, "secondCQs" : i, "score" : child[i] }  for key_ in covidSemanticCQs for child in covidSemanticCQs[key_] for i in child ])
    
QuesSimilar.to_csv("/semanticCQs.csv", sep='|')